In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [99]:
#from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.contrib import rnn

In [125]:
hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
n_hidden = 128


In [126]:
x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

In [131]:
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    #rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
#     tf.variable_scope(reuse=True)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    
    
   # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    #outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32
                                     
    #lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
#     lstm_cell=rnn.BasicLSTMCell(rnn_size,state_is_tuple=True)
#     outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output


In [132]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
      
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))



In [133]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 193.85706958919764
Epoch 1 completed out of 10 loss: 56.350112069398165
Epoch 2 completed out of 10 loss: 37.814688270911574
Epoch 3 completed out of 10 loss: 29.305863903369755
Epoch 4 completed out of 10 loss: 24.440152704250067
Epoch 5 completed out of 10 loss: 20.755572873167694
Epoch 6 completed out of 10 loss: 17.16074212640524
Epoch 7 completed out of 10 loss: 16.213494570693
Epoch 8 completed out of 10 loss: 13.260022854316048
Epoch 9 completed out of 10 loss: 12.659846703754738
Accuracy: 0.9855
